In [49]:
import pandas as pd
import numpy as np

In [50]:
df_initial = pd.read_csv('Zip_Zri_MultiFamilyResidenceRental.csv')
census_df_initial = pd.read_csv('zipcode_census_data_2018_5yr.csv')

In [51]:
include_cities = ['Miami Beach','Miami','North Miami']

df = df_initial.copy()
df = df[df['City'].isin(include_cities)]

census_df = census_df_initial.copy()
census_df = census_df.rename(columns={'geo_id':'RegionID'})

In [52]:
# for i in df['City'].unique():
#     if 'miami' in i.lower():
#         print(i)

In [53]:
# monthly_increase = np.mean(df.iloc[:,2:87].diff(axis=1))
# monthly_increase = round(monthly_increase, 2)

In [54]:
df_melt = df.melt(id_vars = ['RegionID','SizeRank','State','Metro','CountyName', 'City','RegionName'], var_name = "date",value_name="rent")
df_melt['year'] = df_melt['date'].apply(lambda x: x.split('-')[0])
df_melt['month'] = df_melt['date'].apply(lambda x: x.split('-')[1])
for col in ['month','year']:
    df_melt[col] = df_melt[col].astype(int)

In [55]:
dates = df_melt['date'].unique().tolist()
dates.sort()

In [56]:
len(dates)

113

In [57]:
difference_dict = {}

for i in range(1,len(dates)):
    cur_month = dates[i]
    last_month = dates[i - 1]
    
    test = (df[cur_month]-df[last_month]).mean()
    difference_dict[cur_month] = test

In [58]:
def get_prior_month(date_val):
    month = date_val.split('-')[1]
    year = int(date_val.split('-')[0])
    month_map = {'01':'12',
                 '02':'01',
                 '03':'02',
                 '04':'03',
                 '05':'04',
                 '06':'05',
                 '07':'06',
                 '08':'07',
                 '09':'08',
                 '10':'09',
                 '11':'10',
                 '12':'11'
                }
    
    if month == '01':
        return '%s-%s' % (str(year-1),month_map[month])
    else:
        return '%s-%s' % (str(year),month_map[month])

In [59]:
df_melt['difference'] = df_melt['date'].apply(lambda x: difference_dict[x] if x in difference_dict else 0)
df_melt['prior_month'] = df_melt['date'].apply(lambda x: get_prior_month(x))

In [60]:
for col in ['RegionID']:
    census_df[col] = census_df[col].astype(str)
    df_melt[col] = df_melt[col].astype(str)

In [61]:
census_df = census_df[census_df['RegionID'].isin(df_melt['RegionID'].unique().tolist())]

In [62]:
df_melt = pd.merge(df_melt,census_df,on=['RegionID'],how='inner')

In [63]:
df_melt[['rent']].isnull().sum(axis=0)

rent    43
dtype: int64

In [64]:
df_melt = df_melt[df_melt['rent'].isnull()==False]
df_melt = df_melt[df_melt.year == 2018]

In [96]:
for i in df_melt.columns:
    print(i)

RegionID
SizeRank
State
Metro
CountyName
City
RegionName
date
rent
year
month
difference
prior_month
do_date
total_pop
households
male_pop
female_pop
median_age
male_under_5
male_5_to_9
male_10_to_14
male_15_to_17
male_18_to_19
male_20
male_21
male_22_to_24
male_25_to_29
male_30_to_34
male_35_to_39
male_40_to_44
male_45_to_49
male_50_to_54
male_55_to_59
male_65_to_66
male_67_to_69
male_70_to_74
male_75_to_79
male_80_to_84
male_85_and_over
female_under_5
female_5_to_9
female_10_to_14
female_15_to_17
female_18_to_19
female_20
female_21
female_22_to_24
female_25_to_29
female_30_to_34
female_35_to_39
female_40_to_44
female_45_to_49
female_50_to_54
female_55_to_59
female_60_to_61
female_62_to_64
female_65_to_66
female_67_to_69
female_70_to_74
female_75_to_79
female_80_to_84
female_85_and_over
white_pop
population_1_year_and_over
population_3_years_over
pop_5_years_over
pop_15_and_over
pop_16_over
pop_25_years_over
pop_25_64
pop_never_married
pop_now_married
pop_separated
pop_widowed
pop_div

In [66]:
df_melt.head()

,RegionID,SizeRank,State,Metro,CountyName,City,RegionName,date,rent,year,...,management_business_sci_arts_employed,sales_office_employed,in_grades_1_to_4,in_grades_5_to_8,in_grades_9_to_12,in_school,in_undergrad_college,speak_only_english_at_home,speak_spanish_at_home,speak_spanish_at_home_low_english
88,72444,376,FL,Miami-Fort Lauderdale-West Palm Beach,Miami-Dade County,Miami Beach,33141,2018-01,1824.0,2018,...,172.0,107.0,182.0,98.0,57.0,491.0,99.0,NaN,NaN,NaN
89,72444,376,FL,Miami-Fort Lauderdale-West Palm Beach,Miami-Dade County,Miami Beach,33141,2018-02,1799.0,2018,...,172.0,107.0,182.0,98.0,57.0,491.0,99.0,NaN,NaN,NaN
90,72444,376,FL,Miami-Fort Lauderdale-West Palm Beach,Miami-Dade County,Miami Beach,33141,2018-03,1801.0,2018,...,172.0,107.0,182.0,98.0,57.0,491.0,99.0,NaN,NaN,NaN
91,72444,376,FL,Miami-Fort Lauderdale-West Palm Beach,Miami-Dade County,Miami Beach,33141,2018-04,1799.0,2018,...,172.0,107.0,182.0,98.0,57.0,491.0,99.0,NaN,NaN,NaN
92,72444,376,FL,Miami-Fort Lauderdale-West Palm Beach,Miami-Dade County,Miami Beach,33141,2018-05,1792.0,2018,...,172.0,107.0,182.0,98.0,57.0,491.0,99.0,NaN,NaN,NaN


In [116]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
ols = LinearRegression()

from sklearn.ensemble import RandomForestRegressor

rfc = RandomForestRegressor()

In [117]:
df_melt['bachelors_ormore'] = df_melt['bachelors_degree_or_higher_25_64'] / df_melt['total_pop']
df_melt['mean_commute_time'] = df_melt['aggregate_travel_time_to_work'] / df_melt['total_pop']

In [118]:
df_melt['rent'] = df_melt['rent'].astype(float)
target =  df_melt['rent']
feats = df_melt[['total_pop','median_income','median_age','in_undergrad_college']]
for col in feats.columns:
    feats[col] = feats[col].fillna(feats[col].mean())
    feats[col] = feats[col].astype(float)

<ipython-input-118-f2dd12bea04e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feats[col] = feats[col].fillna(feats[col].mean())
<ipython-input-118-f2dd12bea04e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feats[col] = feats[col].astype(float)


In [119]:
Xtrain, Xtest, ytrain, ytest = train_test_split(feats,target)

In [120]:
Xtrain['median_income'].unique()

array([37283., 42183., 35684., 76875.])

In [121]:
ols.fit(Xtrain,ytrain)
print(f"Training R2: {ols.score(Xtrain,ytrain)}")
print(f"Test R2: {ols.score(Xtest,ytest)}")

Training R2: 0.9941676718621162
Test R2: 0.9910999063745894


In [103]:
rfc.fit(Xtrain,ytrain)
rfc.score(Xtest,ytest)

0.9947335292005061

In [114]:
df_melt['in_undergrad_college'].unique()

array([ 99.,   0.,  75., 146.])

In [109]:
len(df_melt)

48